    Описание в data/w3_04_statement-svm-texts.pdf

In [1]:
import pandas as pd
import numpy as np

from sklearn.datasets import fetch_20newsgroups
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def save_answer(name, value):
    with open('data/statement-svm-texts_' + name + '.txt', 'w', encoding='utf-8') as f:
        f.write(value)

In [3]:
newsgroups = fetch_20newsgroups(
                    subset='all', 
                    categories=['alt.atheism', 'sci.space']
             )

In [4]:
X = newsgroups.data
y = newsgroups.target

len(X)

1786

In [5]:
X[0]

'From: 9051467f@levels.unisa.edu.au (The Desert Brat)\nSubject: Re: Keith Schneider - Stealth Poster?\nOrganization: Cured, discharged\nLines: 24\n\nIn article <1pa0f4INNpit@gap.caltech.edu>, keith@cco.caltech.edu (Keith Allan Schneider) writes:\n\n> But really, are you threatened by the motto, or by the people that use it?\n\nEvery time somone writes something and says it is merely describing the norm,\nit is infact re-inforcing that norm upon those programmed not to think for\nthemselves. The motto is dangerous in itself, it tells the world that every\n*true* American is god-fearing, and puts down those who do not fear gods. It\ndoesn\'t need anyone to make it dangerous, it does a good job itself by just\nexisting on your currency.\n\n> keith\n\nThe Desert Brat\n-- \nJohn J McVey, Elc&Eltnc Eng, Whyalla, Uni S Australia,    ________\n9051467f@levels.unisa.edu.au      T.S.A.K.C.            \\/Darwin o\\\nFor replies, mail to whjjm@wh.whyalla.unisa.edu.au      /\\________/\nDisclaimer:

In [6]:
tfidf = TfidfVectorizer()
tfidf.fit(X)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [7]:
X = tfidf.transform(X)

In [8]:
X.shape

(1786, 28382)

In [9]:
len(tfidf.get_feature_names())

28382

In [10]:
cv = KFold(5, shuffle=True, random_state=241)

grid = {'C': np.power(10.0, np.arange(-5, 6))}
clf = SVC(kernel='linear', random_state=241)

gs = GridSearchCV(clf, grid, scoring='accuracy', cv=cv, n_jobs=4)
gs.fit(X, y)

GridSearchCV(cv=KFold(n_splits=5, random_state=241, shuffle=True),
       error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=241, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'C': array([  1.00000e-05,   1.00000e-04,   1.00000e-03,   1.00000e-02,
         1.00000e-01,   1.00000e+00,   1.00000e+01,   1.00000e+02,
         1.00000e+03,   1.00000e+04,   1.00000e+05])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [13]:
gs.best_score_

0.99328107502799556

In [14]:
best_estimator = gs.best_estimator_
best_estimator.fit(X, y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=241, shrinking=True,
  tol=0.001, verbose=False)

In [15]:
answer = list(pd.Series(np.abs(best_estimator.coef_.toarray()[0]), index=tfidf.get_feature_names()).nlargest(10).index)
answer.sort()
answer = ','.join(answer)
answer

'atheism,atheists,bible,god,keith,moon,religion,sci,sky,space'

In [16]:
save_answer('1', answer)